Импорт зависимостей, установление сида для воспроиводимости результатов

In [1]:
import numpy as np
import scipy.stats


ALPHA = 0.05
SEED = 22102025
generator = np.random.default_rng(seed=SEED)

# 1. Моделирование выборок

In [2]:
MEAN_1 = 1.4
MEAN_2 = 1.75
STD_1 = 2.
STD_2 = 1.41
SIZE_1 = 50
SIZE_2 = 30

x1 = generator.normal(loc=MEAN_1, scale=STD_1, size=SIZE_1)
y1 = generator.normal(loc=MEAN_2, scale=STD_2, size=SIZE_2)

## 2. Проверка гипотезы однородности
$ H_0: a_x = a_y $, дисперсии известны

Используется z-test:
$$ z = \dfrac{\overline{x} - \overline{y}}{\sqrt{\dfrac{\sigma_x^2}{n-1} + \dfrac{\sigma_y^2}{m-1}}}, $$
где $\overline{x}, \overline{y}$ — выборочные средние, $n, m$ — размеры выборок, $\sigma_x^2, \sigma_y^2$ — известные дисперсии выборок.
При верности гипотезы $H_0$ эта статистика имеет стандартное нормальное распределение: $z \thicksim N(0, 1)$.

Если гипотеза попадает в промежуток между квантилями уровня $\frac{\alpha}{2}$ и $1 - \frac{\alpha}{2}$, то она принимается, в противном случае отвергается.

In [3]:
z_stat = abs(x1.mean() - y1.mean()) / np.sqrt(STD_1**2 / SIZE_1 + STD_2**2 / SIZE_2)
p_value = scipy.stats.norm.cdf(np.abs(z_stat))

print(ALPHA / 2 < p_value < 1 - ALPHA / 2)

True


In [4]:
p_value

np.float64(0.9688396340172793)

В данном случае получили $p$-value, попадающее в заданный доверительный интервал, $\Rightarrow H_0$ принимается, хотя на самом деле средние не равны. Это можно объяснить малым размером выборок и достаточно близкими значениями средних значениях при генерации выборок.

## 3. Проверка гипотезы о равенстве дисперсий
$ H_0: \sigma_x^2 = \sigma_y^2 $

Используется критерий Фишера:
$$ F = \max\left\{\dfrac{s_1^2}{s_2^2}, \dfrac{s_2^2}{s_1^2}\right\}, $$
где $s_1^2, s_2^2$ — несмещенные (исправленные) выборочные дисперсии. В числителе ставят большую дисперсию, чтобы $  F \ge 1 $.

При справедливости $H_0$ статистика подчиняется распредлению Фишера с $n - 1$ и $m - 1$ степенями свободы.

Гипотеза $H_0$ принимается, если $F \le F_{1-\alpha}(n-1, m-1),$ где последнее выражение — это квантиль распределения Фишера со степенями свободы $(n - 1, m - 1)$ уровня $1 - \alpha$.

In [5]:
var_1 = x1.var(ddof=1) # параметр ddof дает -1 в знаменателе для несмещенности оценки
var_2 = y1.var(ddof=1)

if var_1 > var_2:
    f_stat = var_1 / var_2
    p_value = scipy.stats.f.cdf(f_stat, SIZE_1 - 1, SIZE_2 - 1)
else:
    f_stat = var_2 / var_1
    p_value = scipy.stats.f.cdf(f_stat, SIZE_1 - 1, SIZE_2 - 1)

print(p_value < 1 - ALPHA)

True


In [6]:
p_value

np.float64(0.6351872422407097)

В данном случае получили $p$-value, попадающее в заданный доверительный интервал, $\Rightarrow H_0$ принимается, хотя на самом деле дисперсии не равны. Это также объясняется малым размером выборок.

# 4. Модедирование выборок с равными дисперсиями

In [7]:
MEAN_3 = 0.3
MEAN_4 = 0.6
STD_3 = STD_4 = 0.59
SIZE_3 = 50
SIZE_4 = 30

x2 = generator.normal(loc=MEAN_3, scale=STD_3, size=SIZE_3)
y2 = generator.normal(loc=MEAN_4, scale=STD_4, size=SIZE_4)

## 5. Проверка однородности

$H_0: a_x = a_y $ при равных дисперсиях.

Используется критерий Стьюдента (t-критерий).

Для расчета статистики и $p$-value используется функция из модуля $\verb|scipy|$. Гипотеза принимается, если $p$-value больше уровня значимости.

In [8]:
help(scipy.stats.ttest_ind)

Help on function ttest_ind in module scipy.stats._stats_py:

ttest_ind(a, b, *, axis=0, equal_var=True, nan_policy='propagate', permutations=None, random_state=None, alternative='two-sided', trim=0, method=None, keepdims=False)
    Calculate the T-test for the means of *two independent* samples of scores.

    This is a test for the null hypothesis that 2 independent samples
    have identical average (expected) values. This test assumes that the
    populations have identical variances by default.

    .. deprecated:: 1.17.0
        Use of argument(s) ``{'trim', 'keepdims', 'axis', 'equal_var', 'alternative', 'nan_policy', 'method'}`` by position is deprecated; beginning in
        SciPy 1.17.0, these will be keyword-only. Argument(s) ``{'random_state', 'permutations'}`` are deprecated, whether passed by position or keyword; they will be removed in SciPy 1.17.0. Use ``method`` to perform a permutation test.

    Parameters
    ----------
    a, b : array_like
        The arrays must h

In [9]:
t_stat, p_value = scipy.stats.ttest_ind(x2, y2, equal_var=True)
print(p_value > ALPHA)

True


In [10]:
print(p_value)

0.7874245604981613


Принимается гипотеза $H_0: a_x = a_y$.

# 6. Моделирование выборок малых размеров

In [11]:
MEAN_5 = 12.0
MEAN_6 = 14.0
STD_5 = STD_6 = 4.0
SIZE_5 = 9
SIZE_6 = 7

x3 = generator.normal(loc=MEAN_5, scale=STD_5, size=SIZE_5)
y3 = generator.normal(loc=MEAN_6, scale=STD_6, size=SIZE_6)

## 7. Использование критерия Уилкоксона для проверки гипотезы о статистической однородности

Для проверки гипотезы используется функция из модуля $\verb|scipy|$. Она расчитывает значение статистики и $p$-value, которое затем сравнивается с уровнем значимости для принятия или отвержения гипотезы.

In [12]:
help(scipy.stats.mannwhitneyu)

Help on function mannwhitneyu in module scipy.stats._mannwhitneyu:

mannwhitneyu(x, y, use_continuity=True, alternative='two-sided', axis=0, method='auto', *, nan_policy='propagate', keepdims=False)
    Perform the Mann-Whitney U rank test on two independent samples.

    The Mann-Whitney U test is a nonparametric test of the null hypothesis
    that the distribution underlying sample `x` is the same as the
    distribution underlying sample `y`. It is often used as a test of
    difference in location between distributions.

    Parameters
    ----------
    x, y : array-like
        N-d arrays of samples. The arrays must be broadcastable except along
        the dimension given by `axis`.
    use_continuity : bool, optional
        Whether a continuity correction (1/2) should be applied.
        Default is True when `method` is ``'asymptotic'``; has no effect
        otherwise.
    alternative : {'two-sided', 'less', 'greater'}, optional
        Defines the alternative hypothesis. De

In [13]:
u_stat, p_value = scipy.stats.mannwhitneyu(x3, y3, alternative='two-sided')
print(p_value > ALPHA)

True


In [14]:
p_value

np.float64(0.29912587412587416)

Принимается гипотеза $H_0$.